# Generate Description for your Youtube Video using Steamship's Audio Description Package 

This notebook demonstrates how to create and use the audio-description package.

Transcribe, summarize, and generate post

In [1]:

!pip install steamship===2.2.1



In [3]:


from steamship import Steamship, PackageInstance, File, MimeTypes, Tag
from steamship.base import TaskState

import time

## Constants

#### Using this notebook requires a Steamship API key. If you do not have one, you can create one:



https://app.steamship.com





In [4]:

# The globally unique name of the package. Think of this like an NPM or PyPI package name.
PACKAGE_HANDLE = "audio-description"

# The name that identifies the instance of the package we'll be working with.
INSTANCE_HANDLE = "audio-description-test"


# Authenticate with Steamship

Instantiate the Steamship Client and pass the "API KEY" manually to the Client.

In [5]:


ship = Steamship(api_key="")



# Create a new instance of the package

Use the above instance to create an instance for the package and pass the PACKAGE_HANDLE & INSTANCE_HANDLE

In [6]:
instance = ship.use(package_handle = PACKAGE_HANDLE, instance_handle=INSTANCE_HANDLE)

print(f"""
{'Invocation URL': <20}: {instance.invocation_url}
{'Bearer Token': <20}: {instance.client.config.api_key} 
{'Instance ID': <20}: {instance.id} 
{'Version ID': <20}: {instance.package_version_id} 
{'App ID': <20}: {instance.package_id}
{'Workspace ID': <20}: {instance.workspace_id}
"""
)


Invocation URL      : https://sumanth07.steamship.run/default/audio-description-test/
Bearer Token        : 35739754-99DE-4585-86E4-DD10706414F3 
Instance ID         : 907F240E-FE04-45C7-A33E-BFDD8CC184A9 
Version ID          : 91A060D7-32AA-4903-8C1D-65B7C6F966AD 
App ID              : BFFBD686-AC34-4B26-AF79-600E03E9C350
Workspace ID        : 057BB052-0583-4BA4-968C-1A80620947E3



# Submitting a Youtube Video URL

The audio description package takes youtube video URL as an input.

In the Variable below pass the YT Video Link, for which you want to generate the description

In [7]:


url = "https://youtu.be/0gNauGdOkro"



Now Invoke the "analyze_youtube" function and pass the url.

The POST request triggers the asynchronous transcription and analysis of your video and stores the results in your workspace for future access.

After submitting your POST request you will receive a response that includes a task_id and status key.

The status key shows you the status of your analysis task. It will start with "waiting", and then proceed to "processing", and finally to "completed" or "failed".

In [8]:
# Transcribe and get the summarized output

transcribe_task = instance.invoke("analyze_youtube", url=str(url))

task_id = transcribe_task["task_id"]
status = transcribe_task["status"]

print(
    f"""
task ID: {task_id}
status: {status}
"""
)


task ID: 30AE85D3-1944-476D-A9C0-F98A6751780F
status: waiting



## Retrieving the Results

As your file is being processed the "status" will go from "waiting" to "processing" to "completed" or "failed". You can check in on the progress of your analysis task using the "task_id" by calling the get_status endpoint.

You'll have to make repeated GET requests untill the status converges to "completed" or "failed". Once the status key is set to "completed", you'll see a file key that represents the video description.

To facilitate future file retrieval Steamship will store and index the augmented transcription in your workspace.

In [9]:


retries = 0
while retries <= 100 and status != TaskState.succeeded:
    response = instance.invoke("status", task_id=task_id)
    status = response["status"]
    if status == TaskState.failed:
        print(f"[FAILED] {response['status_message']}")
        break

    print(f"[Try {retries}] Transcription {status}.")
    if status == TaskState.succeeded:
        break
    time.sleep(2)
    retries += 1
    
    
print(response)


[Try 0] Transcription succeeded.
{'status': 'succeeded', 'task_id': '30AE85D3-1944-476D-A9C0-F98A6751780F', 'file': '\n\nReady to learn how Artificial Intelligence (AI) is making its mark on the world of online content creation? Check out my latest video to get the lowdown on this revolutionary technology and what it means for content creators! From understanding AI’s capabilities and limitations to exploring how it can (and can’t) be used to create content, this video is a must-watch for any content creator looking to stay ahead of the curve. #AI #ContentCreation #OnlineContent'}


# Getting the Generated Text from the Response

In [10]:

generated_text = response["file"]
print(generated_text)




Ready to learn how Artificial Intelligence (AI) is making its mark on the world of online content creation? Check out my latest video to get the lowdown on this revolutionary technology and what it means for content creators! From understanding AI’s capabilities and limitations to exploring how it can (and can’t) be used to create content, this video is a must-watch for any content creator looking to stay ahead of the curve. #AI #ContentCreation #OnlineContent
